In [1]:
import torch
import torch.nn as nn
import random
import gym
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros import SuperMarioBrosEnv
from tqdm import tqdm
import pickle 
import gym
import numpy as np
import collections 
import cv2
import matplotlib.pyplot as plt
import time
import datetime
import json
from toolkit.gym_env import *
from toolkit.action_utils import *
import pandas as pd # here if you need it

# models CHANGE FOR YOUR MODEL
from toolkit.train_marlios_rnn import make_env #NOTE your make env may not be here
from toolkit.marlios_rnn import *
##################################

from toolkit.constants import *
from toolkit.train_test_samples import *
import warnings
from toolkit.statistics import *
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
# create the environment (don't change)
mario_env='SuperMarioBros-1-1-v0'
env = gym.make(mario_env)
env = make_env(env, ACTION_SPACE) # (this is defined in my train_marlios_rnn.py, make sure you import yours)

####################
# load the agent (change depending on your model architecture)
agent = DQNAgent(
                state_space=env.observation_space.shape,
                action_space=TEST_SET, #NOTE CHANGE THIS depending on the stats you track
                gamma=0.9,
                max_memory_size=3000, # this doesnt matter here
                batch_size=64, # this doesnt matter here
                lr=0,
                dropout=None,
                exploration_max=0,
                exploration_min=0,
                exploration_decay=0.9995,
                double_dq=True,
                pretrained=True,
                run_id='marlios_rnn_test', #NOTE CHANGE THIS
                n_actions=64,  
                training_stage='test', # your model may not have this parameter, make sure if you're running the rnn its 'train', 'test' or 'val' depending on action space
                device='cpu', # probably fine to leave this
                add_sufficient=True, # your action utils may not require this (or your model)
                hidden_shape=64 # must be the same for what we trained the model on (can reference wandb)
                )

# setup lists of stats we are tracking
num_episodes = 100
total_rewards = []
x_position = []
flags_got, pipe2, pipe3, pipe4 = 0, 0, 0, 0

for ep in tqdm(range(num_episodes)):
    stats = get_stats_run(agent, env) #NOTE: my validate run is coming from statistics.py
    total_rewards.append(stats['total_reward'])
    x_position.append(stats['x_pos'])
    flags_got += stats['flag_get']
    pipe2 += stats['pipe2']
    pipe3 += stats['pipe3']
    pipe4 +=  stats['pipe4']

 32%|███▏      | 32/100 [19:29<41:25, 36.55s/it] 


KeyboardInterrupt: 

In [11]:
def print_statistics(training_stage, total_rewards, x_position, flags, pipes, num_episodes):
    pipe2, pipe3, pipe4 = pipes

    avg_total_rewards = np.mean(total_rewards)
    stddev_rewards = np.std(total_rewards)

    avg_xpos = np.mean(x_position)
    stddev_xpos = np.mean(x_position)

    print("Statistics for ", training_stage)
    print(f"Avg Total Rewards: {avg_total_rewards:.3f}")
    print(f"Std Dev Rewards: {stddev_rewards:.3f}")
    print(f"# of Wins: {flags_got}")
    print(f"% Pipe 1 {100*pipe2/num_episodes:.3f}")
    print(f"% Pipe 2 {100*pipe3/num_episodes:.3f}")
    print(f"% Pipe 3 {100*pipe4/num_episodes:.3f}")

In [12]:
print_statistics(training_stage="TEST (holdout)", 
                 total_rewards=total_rewards,
                 x_position=x_position,
                 flags=flags_got,
                 pipes=(pipe2, pipe3, pipe4),
                 num_episodes=num_episodes
                 )

Statistics for  TEST (holdout)
Avg Total Rewards: 462.260
Std Dev Rewards: 270.286
# of Wins: 0
% Pipe 1 83.000
% Pipe 2 22.000
% Pipe 3 1.000
